In [1]:
from langchain.llms.fake import FakeListLLM
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = load_tools(["python_repl"])
responses = ["Action: Python_REPL\nAction Input: print(2 + 2)", "Final Answer: 4"]
llm = FakeListLLM(responses=responses)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)
agent.run("whats 2 + 2")

Python REPL can execute arbitrary code. Use with caution.




> Entering new AgentExecutor chain...
Action: Python_REPL
Action Input: print(2 + 2)
Observation: 4

Thought:Final Answer: 4

> Finished chain.


'4'

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings  
from dotenv import load_dotenv
load_dotenv()
embeddings = OpenAIEmbeddings()  

text = "This is a sample query."  
query_result = embeddings.embed_query(text)  
print(query_result)
print(len(query_result))

[-0.00915999912004116, 0.006975482333563722, -0.006220088443420029, -0.008765288773894295, -0.026894737480729965, 0.02798359328688594, -0.012930162465271542, -0.004750133105109465, -0.027071675734323043, -0.02719417259459221, 0.010575511668043925, 0.019994113925631805, 0.0032801779996295475, -0.006243907018660537, 0.00383821648386916, 0.0014886699165116313, 0.02052493054905622, -0.0037701631123997358, 0.023124574135499582, -0.01777556940568174, 0.00031240803008799695, -0.0032376444969420023, 0.005658647139267024, 0.012106714970827777, -0.0030147694550866906, 0.0004525556653297457, 0.007948647384938608, -0.02287958227760643, -0.01087494678190617, -0.0031900071136303403, 0.0027255421024726793, 0.002390378660088378, -0.015393697911792167, -0.03495226858363899, -0.01260350572342475, 0.00587641830049822, 0.001714097131509649, -0.005066582085708023, 0.02321984843646161, 0.002017785828471296, 0.032638449762726895, 0.011984219041881202, -0.0022117381816313363, -0.014998987565645303, 0.01524398

In [ ]:
from typing import Any
from langchain.document_loaders import (
    PyPDFLoader, TextLoader,
    UnstructuredWordDocumentLoader,
    UnstructuredEPubLoader
)


class EpubReader(UnstructuredEPubLoader):
    def __init__(self, file_path, **kwargs: Any):
        super().__init__(file_path, **kwargs, mode="elements", strategy="fast")


class DocumentLoaderException(Exception):
    pass


class DocumentLoader(object):
    """Loads in a document with a supported extension."""
    supported_extentions = {
        ".pdf": PyPDFLoader,
        ".txt": TextLoader,
        ".epub": EpubReader,
        ".docx": UnstructuredWordDocumentLoader,
        ".doc": UnstructuredWordDocumentLoader
    }
import logging
import pathlib
from langchain.schema import Document
def load_document(temp_filepath: str) -> list[Document]:
    """Load a file and return it as a list of documents."""
    ext = pathlib.Path(temp_filepath).suffix
    loader = DocumentLoader.supported_extentions.get(ext)
    if not loader:
        raise DocumentLoaderException(
            f"Invalid extension type {ext}, cannot load this type of file"
        )
    loader = loader(temp_filepath)
    docs = loader.load()
    logging.info(docs)
    return docs

from langchain.embeddings import HuggingFaceEmbeddings 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.schema import Document, BaseRetriever
def configure_retriever(docs: list[Document]) -> BaseRetriever:
    """Retriever to use."""
    # Split each document documents:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    # Create embeddings and store in vectordb:
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    # Single call to the huggingface model with all texts:
    vectordb = DocArrayInMemorySearch.from_documents(splits, embeddings)
    # Define retriever:
    return vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 2, "fetch_k": 4})